In [234]:
import math

import pandas
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from MyCode import relabeling_fairlearn
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

In [235]:
round_value = 10

In [236]:
if True:
    from fairlearn.datasets import fetch_adult

    dataset = fetch_adult(as_frame=True)
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    df['>50k'] = (dataset.target == '>50K') * 1


    le = preprocessing.LabelEncoder()
    df['sex'] = le.fit_transform(df['sex'])

    onehot = ['workclass', 'education', 'marital-status', 'occupation', 'marital-status', 'occupation', 'relationship', 'race']
    df = pd.get_dummies(df,prefix=onehot, columns = onehot, drop_first=True)
    tmp = df['sex'].to_list()
    for i in range(0,len(tmp)):
        if tmp[i] == 1:
            tmp[i] = 0
        else:
            tmp[i] = 1
    df['sex'] = tmp
    y = df[">50k"]
    sensitive = df['sex']
    X = df.loc[:, ~df.columns.isin(['sex', '>50k', 'native-country'])]
    print(relabeling_fairlearn.discrimination_dataset(y, sensitive))


0.19451574596420296


In [237]:
if False:
    pd.set_option('max_columns', None)
    FILE_NAME = "relab_Lawsuit"

    # src: https://www.kaggle.com/hjmjerry/gender-discrimination
    df = pandas.read_csv("dataset_perso/Lawsuit.csv")
    del df['ID']
    salary_mean = np.mean(np.array(df["Sal94"].tolist(), float))
    for i in range(0,len(df)):
        if df.at[i, "Sal94"] >= salary_mean:
            df.at[i, "Salary_mean"] = 1
        else:
            df.at[i, "Salary_mean"] = 0
    del df["Sal94"]
    del df["Sal95"]
    col = ['Rank', 'Dept']
    df = df.drop(['Prate', 'Exper'], axis=1)
    df = pd.get_dummies(df, columns=col)
    for i in range(0,len(df)):
        if df.at[i, "Gender"] == 1:
            df.at[i, "Gender"] = 0
        else:
            df.at[i, "Gender"] = 1
    X = df.loc[:, ~df.columns.isin(['Gender', 'Salary_mean'])]
    y = df['Salary_mean']
    sensitive = df['Gender']

In [238]:
if False:
    pd.set_option('max_columns', None)
    FILE_NAME = "HR"

    # src: https://www.kaggle.com/hjmjerry/gender-discrimination
    df = pandas.read_csv("dataset_perso/HRDataset_v14.csv")
    del df['Employee_Name']
    del df['EmpID']
    salary_mean = np.mean(np.array(df["Salary"].tolist(), float))
    print(f"Mean: {salary_mean}")
    for i in range(0,len(df)):
        if df.at[i, "Salary"] >= salary_mean:
            df.at[i, "Salary_mean"] = 1
        else:
            df.at[i, "Salary_mean"] = 0
    del df["Salary"]

    for i in range(0,len(df)):
        df.at[i, "Absences"] = df.at[i, "Absences"] / 5
    for i in range(0,len(df)):
        df.at[i, "EngagementSurvey"] = int(df.at[i, "EngagementSurvey"])
    for i in range(0,len(df)):
        if df.at[i, "HispanicLatino"] == 'Yes' or df.at[i, "HispanicLatino"] == 'yes' :
            df.at[i, "HispanicLatino"] = 1
        elif df.at[i, "HispanicLatino"] == 'No' or df.at[i, "HispanicLatino"] == 'no' :
            df.at[i, "HispanicLatino"] = 0
    col = ['EmpStatusID', 'PerfScoreID', 'Position', 'MaritalDesc', 'CitizenDesc', 'RaceDesc', 'Department', 'PerformanceScore', 'EmpSatisfaction', 'Absences']
    df = df.drop(['MarriedID', 'MaritalStatusID', 'Zip', 'DOB', 'Sex', 'DateofHire','DateofTermination', 'TermReason', 'EmploymentStatus', 'ManagerName', 'ManagerID', 'EngagementSurvey', 'LastPerformanceReview_Date', 'DaysLateLast30', 'RecruitmentSource', 'State', 'DeptID', 'PositionID', 'SpecialProjectsCount'], axis=1)
    df = pd.get_dummies(df, columns=col)
    for col in df:
        if len(df[col].unique()) > 2:
            print(f'{col}: {df[col].unique()}')
    for i in range(0,len(df)):
        if df.at[i, "GenderID"] == 1:
            df.at[i, "GenderID"] = 0
        else:
            df.at[i, "GenderID"] = 1
    X = df.loc[:, ~df.columns.isin(['Gender', 'Salary_mean'])]
    y = df['Salary_mean']
    sensitive = df['GenderID']
    print(relabeling_fairlearn.discrimination_dataset(y, sensitive))

In [239]:
if False:
    data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
    data_url = "dataset_perso/default of credit card clients.xls"
    dataset = pd.read_excel(io=data_url, header=1).drop(columns=['ID']).rename(columns={'PAY_0':'PAY_1'})

    tmp = dataset['SEX'].to_list()
    for i in range(0,len(tmp)):
        if tmp[i] == 1:
            tmp[i] = 0
        elif tmp[i] == 2:
            tmp[i] = 1
        else:
            raise Exception(i)

    dataset["SEX"] = tmp
    sensitive = dataset["SEX"]
    y = dataset["default payment next month"]
    X = dataset.loc[:, ~dataset.columns.isin(['SEX', 'default payment next month'])]
    categorical_features = ['EDUCATION', 'MARRIAGE','PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
    for col in categorical_features:
        dataset[col] = dataset[col].astype('category')

In [240]:
def train_test_split(random_state, X, y, sensitive):
    index_train = list(X.sample(frac=0.8, random_state=random_state).index)
    index_test = list(X.drop(index=index_train).index)

    X_train = X.drop(index=index_test).to_numpy()
    y_train = y.drop(index=index_test).to_numpy()
    sensitive_train = sensitive.drop(index=index_test).to_numpy()

    X_test = X.drop(index=index_train).to_numpy()
    y_test = list(y.drop(index=index_train).to_numpy())
    sensitive_test = sensitive.drop(index=index_train).to_numpy()

    return X_train, X_test, y_train, y_test, sensitive_train, sensitive_test

X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(1, X, y, sensitive)
print(relabeling_fairlearn.discrimination_dataset(y_train, sensitive_train))

0.1925655508416967


# Basic usage

In [241]:
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, list(y_train))
y_pred = clf.predict(X_train)
accuracy = round(accuracy_score(y_train, y_pred),round_value)
print("Accuracy of the prediction: ", accuracy)
discrimination= round(relabeling_fairlearn.discrimination_dataset(y_pred, sensitive_train), round_value)
print("Discrimination of classifier on the predidiction: ", discrimination)

Accuracy of the prediction:  0.8396120182
Discrimination of classifier on the predidiction:  0.1882659298


In [242]:
"""
# Raises an exception
sensitive = list(sensitive)
sensitive[0] = 2
sensitive = np.asarray(sensitive)
print(sensitive)
print(np.unique(sensitive))
"""
threshold = 0.1

In [243]:
relabeling_fairlearn.relabeling(clf, X_train, y_train, y_pred, sensitive_train, threshold)
y_pred_relab = clf.predict(X_train)
accuracy_relab = round(accuracy_score(y_train, y_pred_relab),round_value)
print("Accuracy of the prediction: ", accuracy_relab)
discrimination_relab = round(relabeling_fairlearn.discrimination_dataset(y_pred_relab, sensitive_train), round_value)
print("Discrimination of classifier on the prediction: ", discrimination_relab)

Accuracy of the prediction:  0.8047294876
Discrimination of classifier on the prediction:  0.0353824652


# Detailed operations

In [244]:
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, list(y_train))
y_pred = clf.predict(X_train)
accuracy = round(accuracy_score(y_train, y_pred), round_value)
print("Accuracy of the prediction: ", accuracy)
discrimination= round(relabeling_fairlearn.discrimination_dataset(y_pred, sensitive_train), round_value)
print("Discrimination of classifier on the prediction: ", discrimination)

Accuracy of the prediction:  0.8396120182
Discrimination of classifier on the prediction:  0.1882659298


In [245]:
threshold = 0

In [246]:
i = list()
cnt = np.unique(sensitive, return_counts=True)[1]
relabeling_fairlearn.get_leaves_candidates(clf, X_train, y_train, sensitive_train, cnt, len(y), i)
for e in i:
    print(e)

Path: format -> (feature, type, node id)
((67, 'left', 0), (3, 'left', 1), (2, 'left', 2), (-2, 'leaf', 3)) 
node_id: 3 
The effect of relabeling the leaf on accuracy: -0.33708693337701157
The effect of relabeling the leaf on discrimination: -0.3628553529166087 
-(9384 + 552) / 16192 + (7910 + 278) / 32650= -0.36285535291660864 
ratio: 1.0764444331361154 
contingency table: 
[0.19212972441750953, 0.01130174849514762]
[0.16195078006633634, 0.005691822611686663]
transactions: [0, 4, 5, 6, 8, 11, 13, 14, 16, 17, 21, 23, 25, 28, 31, 32, 35, 36, 38, 39, 41, 42, 48, 51, 56, 58, 61, 64, 67, 69, 72, 75, 77, 78, 79, 80, 81, 83, 84, 85, 86, 89, 90, 92, 93, 95, 96, 97, 101, 102, 105, 107, 112, 113, 115, 116, 117, 118, 119, 120, 128, 137, 140, 141, 143, 145, 146, 147, 148, 149, 160, 161, 164, 165, 166, 169, 170, 175, 177, 182, 184, 185, 187, 189, 190, 191, 195, 199, 200, 201, 203, 204, 209, 212, 215, 216, 218, 221, 222, 223, 224, 230, 233, 235, 237, 238, 239, 241, 246, 249, 250, 251, 252, 255, 258

In [247]:
leafs_relab = relabeling_fairlearn.leaves_to_relabel(clf, X_train, y_train, y_pred, sensitive_train, threshold)

sum_acc = 0
sum_disc = 0
for l in leafs_relab:
    print(l)
    print()
    sum_acc += l.acc
    sum_disc += l.disc
sum_acc = round(sum_acc, round_value)
sum_disc = round(sum_disc, round_value)

Path: format -> (feature, type, node id)
((67, 'left', 0), (3, 'left', 1), (2, 'right', 2), (-2, 'leaf', 4)) 
node_id: 4 
The effect of relabeling the leaf on accuracy: -0.07130060910068076
The effect of relabeling the leaf on discrimination: -0.1324337999574769 
-(2082 + 602) / 12903 + (1642 + 336) / 26171= -0.13243379995747684 
ratio: 1.8574006818156685 
contingency table: 
[0.053283513333674565, 0.015406664278036546]
[0.0420228284792957, 0.008599068434252956]
transactions: [18, 26, 46, 47, 49, 63, 71, 73, 74, 94, 111, 121, 122, 125, 130, 131, 138, 142, 144, 150, 154, 155, 167, 179, 197, 198, 202, 205, 213, 240, 242, 248, 270, 275, 284, 287, 295, 301, 304, 305, 313, 315, 323, 329, 332, 340, 344, 361, 364, 365, 370, 383, 384, 389, 397, 407, 420, 425, 426, 427, 431, 438, 461, 470, 488, 494, 511, 516, 519, 543, 549, 570, 574, 582, 587, 591, 605, 608, 609, 611, 615, 617, 619, 633, 639, 646, 648, 649, 650, 659, 666, 680, 689, 695, 702, 719, 723, 724, 732, 735, 760, 780, 802, 810, 813, 832

In [248]:
relabeling_fairlearn.relabeling(clf, X_train, y_train, y_pred, sensitive_train, threshold)
y_pred_relab = clf.predict(X_train)
accuracy_relab = round(accuracy_score(y_train, y_pred_relab), round_value)
discrimination_relab = round(relabeling_fairlearn.discrimination_dataset(y_pred_relab, sensitive_train), round_value)
new_acc= round(accuracy+sum_acc, round_value)
new_disc = round(discrimination+sum_disc, round_value)
print(f"Accuracy:\n"
    f"    Before      : {accuracy}\n"
    f"    After       : {accuracy_relab}\n"
    f"    Expected    : {new_acc}\n"
    f"    Leafs       : {sum_acc}\n"
    f"    Difference  : {round(-accuracy+accuracy_relab, round_value)}\n"
    f"    Check       : {sum_acc == round(-accuracy+accuracy_relab, round_value)}")
print(f"Discrimination:\n"
    f"    Before      : {discrimination}\n"
    f"    After       : {discrimination_relab}\n"
    f"    Expected    : {new_disc}\n"
    f"    Leafs       : {sum_disc }\n"
    f"    Difference  : {round(-discrimination+discrimination_relab, round_value)}\n"
    f"    Check       : {sum_disc == round(-discrimination+discrimination_relab, round_value)}")

Accuracy:
    Before      : 0.8396120182
    After       : 0.7334288785
    Expected    : 0.7334288785
    Leafs       : -0.1061831396
    Difference  : -0.1061831396
    Check       : True
Discrimination:
    Before      : 0.1882659298
    After       : -0.0970513347
    Expected    : -0.0970513346
    Leafs       : -0.2853172645
    Difference  : -0.2853172645
    Check       : True
